![alt text for screen readers](https://intro-to-btt-using-python-assets.s3.amazonaws.com/bladesight_logo_horizontal_ORIGINAL.jpg).
## Chapter 3: Angle of Arrival 

## Dependencies

In [ ]:
# Run this cell if you have not installed the `bladesight` package yet
%pip install bladesight
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If plotly is not installed
%pip install plotly
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If Numba is not installed
%pip install numba
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

## Imports

In [ ]:
from bladesight import Datasets
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from numba import njit
import pandas as pd

In [ ]:
# Load the dataset
dataset = Datasets["data/intro_to_btt/intro_to_btt_ch03"]

In [ ]:
## Get the OPR zero-crossing table
df_opr_zero_crossings = dataset['table/du_toit_2017_test_1_opr_zero_crossings']
df_opr_zero_crossings

## Calculate and plot the shaft speed

In [ ]:
df_opr_zero_crossings["period"] = df_opr_zero_crossings["time"].diff().bfill()
df_opr_zero_crossings["shaft_speed"] = 1 / df_opr_zero_crossings["period"] * 60
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_opr_zero_crossings["time"],
        y=df_opr_zero_crossings["shaft_speed"],
        mode="markers",
        name="Shaft speed",
    )
)

fig.update_layout(
    title="Shaft speed",
    xaxis_title="Time [s]",
    yaxis_title="Shaft speed [rpm]",
    legend_title="",
)

## Algorithm

In [ ]:
from numba import njit
import numpy as np

@njit
def calculate_aoas(
    arr_opr_zero_crossing : np.ndarray, 
    arr_probe_toas : np.ndarray
):
    """
    This function calculates the angle of arrival of 
    each ToA value relative to the revolution in 
    which it occurs.

    Args:
        arr_opr_zero_crossing (np.array): An array of 
            OPR zero-crossing times. 
        arr_probe_toas (np.array): An array of 
            ToA values.

    Returns:
        np.array: A matrix of AoA values. Each row in the 
            matrix corresponds to a ToA value. The columns 
            are:
            0: The revolution number
            1: The zero crossing time at the start of the revolution
            2: The zero crossing time at the end of the revolution
            3: The angular velocity of the revolution
            4: The ToA
            5: The AoA of the ToA value
    """
    num_toas = len(arr_probe_toas)
    AoA_matrix = np.zeros((num_toas, 6))

    AoA_matrix[:, 0] = -1

    current_zero_crossing_start = arr_opr_zero_crossing[0]
    current_zero_crossing_end = arr_opr_zero_crossing[1]
    current_n = 0

    for i, toa in enumerate(arr_probe_toas):

        while toa > current_zero_crossing_end:
            current_n += 1
            if current_n >= (len(arr_opr_zero_crossing) - 1):
                break
            current_zero_crossing_start = arr_opr_zero_crossing[current_n]
            current_zero_crossing_end = arr_opr_zero_crossing[current_n + 1]

        if current_n >= (len(arr_opr_zero_crossing) - 1):
            break

        if toa > current_zero_crossing_start:
            AoA_matrix[i, 0] = current_n
            AoA_matrix[i, 1] = current_zero_crossing_start
            AoA_matrix[i, 2] = current_zero_crossing_end
            Omega = 2 * np.pi / (
                current_zero_crossing_end 
                - current_zero_crossing_start
            )
            AoA_matrix[i, 3] = Omega
            AoA_matrix[i, 4] = toa
            AoA_matrix[i, 5] = Omega * (
                toa 
                - current_zero_crossing_start
            )

    return AoA_matrix

## Example usage

In [ ]:
# Get the zero-crossing times and the first proximity probe's ToA values
dataset = Datasets["data/intro_to_btt/intro_to_btt_ch03"]
df_opr_zero_crossings = dataset['table/du_toit_2017_test_1_opr_zero_crossings']
df_probe_toas = dataset['table/du_toit_2017_test_1_prox_1_toas']
AoA_matrix = calculate_aoas(
    df_opr_zero_crossings["time"].to_numpy(),
    df_probe_toas["time"].to_numpy()
)
df_AoA = pd.DataFrame(
    AoA_matrix, 
    columns=[
        "n",
        "n_start_time",
        "n_end_time",
        "Omega",
        "ToA",
        "AoA"
    ]
)
df_AoA.head(10)

## Drop the ToAs that could not be allocated

In [ ]:
df_AoA = df_AoA[df_AoA["n"] != -1]
df_AoA.reset_index(inplace=True, drop=True)

## Plot results

In [ ]:
# Plot the AoA values
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Just plot the first blade, i.e. take every 5th value
df_aoa_1 = df_AoA.iloc[::5]
fig.add_trace(go.Scattergl(x=df_aoa_1["ToA"], y=df_aoa_1["AoA"], name="AoA"), secondary_y=False)
fig.add_trace(go.Scattergl(x=df_aoa_1["n_start_time"], y=df_aoa_1["Omega"]*60/(2*np.pi), name="Shaft Speed"), secondary_y=True)

fig.update_layout(
    title="AoA of blade 1 arriving at probe 1",
    xaxis_title="Time [s]",
    yaxis_title="AoA [rad]",
    legend_title="",
    yaxis2_title="Shaft speed [RPM]",
)


fig.show()